In [87]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('f1_2019_to_2022_all_drivers_all_data.csv',low_memory=False) # it's for 2019 to 2022 <- Train 

#Change to numerical values
df['LapTime'] = pd.to_timedelta(df['LapTime']).dt.total_seconds()

columns_to_check = ['Position', 'IsPersonalBest', 'TrackStatus']
df = df.dropna(subset=columns_to_check)


# Change strings to integer
df['Sector1Time'] = pd.to_timedelta(df['Sector1Time']).dt.total_seconds()
df['Sector2Time'] = pd.to_timedelta(df['Sector2Time']).dt.total_seconds()
df['Sector3Time'] = pd.to_timedelta(df['Sector3Time']).dt.total_seconds()


df['Rainfall'] = df['Rainfall'].astype(int) # Encode Rainfall To integer
df['FreshTyre'] = df['FreshTyre'].astype(int) # Encode FreshTyre To integer
df['IsAccurate'] = df['IsAccurate'].astype(int)
df["IsPersonalBest"] = df['IsPersonalBest'].astype(int)



#Drop non-related features 
columns_to_drop = [
    'Time', 'LapStartTime',
    'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
    'PitOutTime', 'PitInTime',
    'LapStartDate', 'Deleted', 'DeletedReason',
    'FastF1Generated'
]
df = df.drop(columns=columns_to_drop)


Q1 = df['LapTime'].quantile(0.25)
Q3 = df['LapTime'].quantile(0.75)
IQR = Q3 - Q1

# Determine bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = df[(df['LapTime'] < lower_bound) | (df['LapTime'] > upper_bound)]

#  Remove outliers from the DataFrame

df = df[(df['LapTime'] >= lower_bound) & (df['LapTime'] <= upper_bound)]

#One hot encoding
df = pd.get_dummies(df, columns=['Driver', 'Circuit','Compound','Team','TrackStatus',])

# In case of Standardization
numeric_features = ['SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'AirTemp', 'Humidity', 'Pressure', 'TrackTemp', 'WindDirection', 'WindSpeed',]

df = df.dropna(subset=['LapTime'])

X = df.drop(['LapTime'], axis=1) # Exclude response variable ('LapTime') 

y = df['LapTime']

print(X.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79034 entries, 0 to 82417
Data columns (total 124 columns):
 #    Column                               Dtype  
---   ------                               -----  
 0    DriverNumber                         int64  
 1    LapNumber                            float64
 2    Stint                                float64
 3    Sector1Time                          float64
 4    Sector2Time                          float64
 5    Sector3Time                          float64
 6    SpeedI1                              float64
 7    SpeedI2                              float64
 8    SpeedFL                              float64
 9    SpeedST                              float64
 10   IsPersonalBest                       int64  
 11   TyreLife                             float64
 12   FreshTyre                            int64  
 13   Position                             float64
 14   IsAccurate                           int64  
 15   AirTemp          

In [88]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=42)


X_train = X_train.drop(['TrackTemp'], axis= 1)


xgb.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [90]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(xgb, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Mean CV RMSE: {np.mean(np.sqrt(-scores))}")


Mean CV RMSE: 2.673095050639977
